# Cleaning NYC Weather Data to populate google cloud sql (postgresm)

In [1]:
import pandas as pd
import numpy as np

In [3]:
weather_df = pd.read_csv('../Datasets/NYC Weather Raw.csv')

C:\Users\imhar\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (5,7,8,11,14,30,31,33,41,42,43,49,51,75,89,123) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
weather_df['DATE'] = pd.to_datetime(weather_df['DATE'])
weather_df.head()

,STATION,DATE,REPORT_TYPE,SOURCE,AWND,BackupDirection,BackupDistance,BackupDistanceUnit,BackupElements,BackupElevation,...,ShortDurationPrecipitationValue060,ShortDurationPrecipitationValue080,ShortDurationPrecipitationValue100,ShortDurationPrecipitationValue120,ShortDurationPrecipitationValue150,ShortDurationPrecipitationValue180,Sunrise,Sunset,TStorms,WindEquipmentChangeDate
0,72505394728,2013-01-01 00:51:00,FM-15,7,NaN,ESE,0.5,mi,SNOW,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2006-09-18
1,72505394728,2013-01-01 01:51:00,FM-15,7,NaN,ESE,0.5,mi,SNOW,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2006-09-18
2,72505394728,2013-01-01 02:51:00,FM-15,7,NaN,ESE,0.5,mi,SNOW,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2006-09-18
3,72505394728,2013-01-01 03:51:00,FM-15,7,NaN,ESE,0.5,mi,SNOW,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2006-09-18
4,72505394728,2013-01-01 04:51:00,FM-15,7,NaN,ESE,0.5,mi,SNOW,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2006-09-18


In [5]:
weather_2018_df = weather_df[(weather_df['DATE'] >= '2018-01-01') & (weather_df['DATE'] < '2018-06-01')]
weather_2018_df.head()

,STATION,DATE,REPORT_TYPE,SOURCE,AWND,BackupDirection,BackupDistance,BackupDistanceUnit,BackupElements,BackupElevation,...,ShortDurationPrecipitationValue060,ShortDurationPrecipitationValue080,ShortDurationPrecipitationValue100,ShortDurationPrecipitationValue120,ShortDurationPrecipitationValue150,ShortDurationPrecipitationValue180,Sunrise,Sunset,TStorms,WindEquipmentChangeDate
57038,72505394728,2018-01-01 00:51:00,FM-15,7,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57039,72505394728,2018-01-01 01:51:00,FM-15,7,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57040,72505394728,2018-01-01 02:51:00,FM-15,7,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57041,72505394728,2018-01-01 03:51:00,FM-15,7,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57042,72505394728,2018-01-01 04:51:00,FM-15,7,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# Get desired columns
new_cols = {
    'DATE': 'Date',
    'HourlyPresentWeatherType': 'H_PresentWeatherType',
    'HourlySkyConditions': 'H_SkyConditions',
    'HourlyDryBulbTemperature': 'H_DryBulbTemperature',
    'HourlyDewPointTemperature': 'H_DewPointTemperature',
    'HourlyWetBulbTemperature': 'H_WetBulbTemperature',
    'HourlyPrecipitation': 'H_Precipitation',
    'HourlyRelativeHumidity': 'H_RelativeHumidity',
    'HourlyVisibility': 'H_Visibility',
    'HourlyWindSpeed': 'H_WindSpeed',
    'Sunrise': 'Sunrise',
    'Sunset': 'Sunset',
}
cols = list(new_cols.keys())
weather_2018_df = weather_2018_df[cols]
weather_2018_df.rename(new_cols,axis=1,inplace=True)
weather_2018_df

,Date,H_PresentWeatherType,H_SkyConditions,H_DryBulbTemperature,H_DewPointTemperature,H_WetBulbTemperature,H_Precipitation,H_RelativeHumidity,H_Visibility,H_WindSpeed,Sunrise,Sunset
57038,2018-01-01 00:51:00,NaN,CLR:00,9,-4,7.0,0.00,55.0,10.00,9.0,NaN,NaN
57039,2018-01-01 01:51:00,NaN,CLR:00,9,-4,7.0,0.00,55.0,10.00,10.0,NaN,NaN
57040,2018-01-01 02:51:00,NaN,CLR:00,8,-4,6.0,0.00,57.0,10.00,10.0,NaN,NaN
57041,2018-01-01 03:51:00,NaN,CLR:00,8,-5,6.0,0.00,55.0,10.00,7.0,NaN,NaN
57042,2018-01-01 04:51:00,NaN,CLR:00,8,-5,6.0,0.00,55.0,10.00,9.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
62146,2018-05-31 21:51:00,BR:1 ||,OVC:08 3,65,64,64.0,0.00,97.0,3.00,0.0,NaN,NaN
62147,2018-05-31 22:51:00,BR:1 ||,OVC:08 3,66,64,65.0,0.00,93.0,3.00,0.0,NaN,NaN
62148,2018-05-31 23:51:00,BR:1 ||,OVC:08 2,65,64,64.0,0.00,97.0,3.00,0.0,NaN,NaN
62149,2018-05-31 23:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,427.0,1920.0


In [7]:
list(new_cols.values())

['Date',
 'H_PresentWeatherType',
 'H_SkyConditions',
 'H_DryBulbTemperature',
 'H_DewPointTemperature',
 'H_WetBulbTemperature',
 'H_Precipitation',
 'H_RelativeHumidity',
 'H_Visibility',
 'H_WindSpeed',
 'Sunrise',
 'Sunset']

In [8]:
weather_2018_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5113 entries, 57038 to 62150
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Date                   5113 non-null   datetime64[ns]
 1   H_PresentWeatherType   1661 non-null   object        
 2   H_SkyConditions        4930 non-null   object        
 3   H_DryBulbTemperature   4957 non-null   object        
 4   H_DewPointTemperature  4957 non-null   object        
 5   H_WetBulbTemperature   4952 non-null   float64       
 6   H_Precipitation        4294 non-null   object        
 7   H_RelativeHumidity     4957 non-null   float64       
 8   H_Visibility           4957 non-null   object        
 9   H_WindSpeed            4743 non-null   float64       
 10  Sunrise                151 non-null    float64       
 11  Sunset                 151 non-null    float64       
dtypes: datetime64[ns](1), float64(5), object(6)
memory usage:

## Cleaning up non-float values

In [9]:
weather_2018_df.sample(20)

,Date,H_PresentWeatherType,H_SkyConditions,H_DryBulbTemperature,H_DewPointTemperature,H_WetBulbTemperature,H_Precipitation,H_RelativeHumidity,H_Visibility,H_WindSpeed,Sunrise,Sunset
58543,2018-02-14 02:51:00,NaN,BKN:07 35 BKN:07 43,36,28,33.0,0.00,73.0,10.00,3.0,NaN,NaN
57340,2018-01-11 22:51:00,NaN,SCT:04 29 OVC:08 34,51,48,49.0,0.00,89.0,10.00,3.0,NaN,NaN
57707,2018-01-22 01:51:00,NaN,BKN:07 50 OVC:08 60,45,37,41.0,T,74.0,10.00,3.0,NaN,NaN
58723,2018-02-19 06:51:00,NaN,CLR:00,40,30,36.0,0.00,68.0,10.00,0.0,NaN,NaN
61242,2018-05-07 14:51:00,NaN,FEW:02 75 SCT:04 90,75,46,59.0,0.00,36.0,10.00,6.0,NaN,NaN
57757,2018-01-23 08:00:00,BR:1 ||,OVC:08 5,51,50,50.0,NaN,96.0,4.00,7.0,NaN,NaN
58011,2018-01-31 09:51:00,NaN,CLR:00,21,3,17.0,0.00,45.0,10.00,0.0,NaN,NaN
59962,2018-03-28 16:51:00,NaN,FEW:02 55 OVC:08 70,51,40,46.0,0.00,66.0,8.00,5.0,NaN,NaN
61164,2018-05-04 17:51:00,NaN,FEW:02 120,82,60,68.0,0.00,47.0,10.00,0.0,NaN,NaN
61965,2018-05-27 09:51:00,BR:1 ||,OVC:08 6,61,60,60.0,T,97.0,4.00,7.0,NaN,NaN


In [10]:
weather_2018_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5113 entries, 57038 to 62150
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Date                   5113 non-null   datetime64[ns]
 1   H_PresentWeatherType   1661 non-null   object        
 2   H_SkyConditions        4930 non-null   object        
 3   H_DryBulbTemperature   4957 non-null   object        
 4   H_DewPointTemperature  4957 non-null   object        
 5   H_WetBulbTemperature   4952 non-null   float64       
 6   H_Precipitation        4294 non-null   object        
 7   H_RelativeHumidity     4957 non-null   float64       
 8   H_Visibility           4957 non-null   object        
 9   H_WindSpeed            4743 non-null   float64       
 10  Sunrise                151 non-null    float64       
 11  Sunset                 151 non-null    float64       
dtypes: datetime64[ns](1), float64(5), object(6)
memory usage:

In [11]:
T = {'T': '0.005'}

In [12]:
def check_str(x):
    try:
        float(x)
    except:
        return x

In [13]:
def temp_func(x):
    try:
        float(x)
        return x
    except:    
        if x == 'T':
            return '0.005' 
        if x[-1] == 's':
            return x[:-1]

In [14]:
clean_col = 'H_Precipitation'
copy_df = weather_2018_df.copy()
# copy[clean_col].fillna('0',inplace=True)
# copy[clean_col] = copy[clean_col].apply(temp_func)
# copy[clean_col] = copy[clean_col].astype('float64')
# copy[clean_col].unique()

In [18]:
weather_2018_df = copy_df.copy()
# clean H_DryBulbTemperature, H_DewPointTemperature
# midnight values are nan, forward fill with last hour's temperature
clean_col = ['H_DryBulbTemperature', 'H_DewPointTemperature', 'H_WetBulbTemperature']
# weather_2018_df[weather_2018_df[clean_col].isnull() == True]
for c in clean_col:
    weather_2018_df[c] = weather_2018_df[c].fillna(method='ffill')
    weather_2018_df[c] = weather_2018_df[c].apply(temp_func)
    print(c)
    print(weather_2018_df[c].unique())
    for i in weather_2018_df[c].unique():
        if check_str(i) != None:
            print(i)
    weather_2018_df[c] = weather_2018_df[c].astype('float64')
weather_2018_df.head()

H_DryBulbTemperature
[9.0 8.0 7.0 11.0 12.0 14.0 15.0 17.0 18.0 16.0 13.0 21.0 23.0 25.0 26.0
 24.0 22.0 20.0 19.0 27.0 29.0 28.0 10.0 6.0 5.0 30.0 31.0 33.0 34.0 37.0
 40.0 42.0 43.0 41.0 39.0 36.0 35.0 32.0 38.0 44.0 45.0 47.0 50.0 51.0
 53.0 49.0 46.0 52.0 '49' '48' '47' '46' '45' '44' '52' '57' '59' '58'
 '56' '55' '60' '61' '54' '53' '51' '36' '35' '30' '28' '29' '32' '33'
 '31' '25' '24' '23' '22' '21' '20' '19' '17' '16' '15' '18' '26' '27'
 '34' '37' '38' '39' '40' '41' '42' '43' '50' '63' '65' '66' '67' '64'
 '62' '72' '74' '75' '77' '78' '73' '69' '80' '81' '82' '68' '71' '70'
 '76' '79' '83' '85' '87' '88' '89' '84' '90' '92' '91' '86']
H_DewPointTemperature
[ -4.  -5.  -3.  -6.  -2.   0.   2.   3.   1.   4.   5.   6.   7.  -1.
  11.  12.   8.   9.  13.  18.  19.  20.  21.  17.  16.  14.  10.  -7.
  -8.  -9. -10. -12. -13. -11.  22.  23.  24.  25.  26.  27.  28.  30.
  31.  32.  33.  34.  36.  37.  35.  39.  43.  42.  44.  41.  46.  47.
  48.  49.  45.  50.  56.  57.  55.  5

,Date,H_PresentWeatherType,H_SkyConditions,H_DryBulbTemperature,H_DewPointTemperature,H_WetBulbTemperature,H_Precipitation,H_RelativeHumidity,H_Visibility,H_WindSpeed,Sunrise,Sunset
57038,2018-01-01 00:51:00,NaN,CLR:00,9.0,-4.0,7.0,0.00,55.0,10.00,9.0,NaN,NaN
57039,2018-01-01 01:51:00,NaN,CLR:00,9.0,-4.0,7.0,0.00,55.0,10.00,10.0,NaN,NaN
57040,2018-01-01 02:51:00,NaN,CLR:00,8.0,-4.0,6.0,0.00,57.0,10.00,10.0,NaN,NaN
57041,2018-01-01 03:51:00,NaN,CLR:00,8.0,-5.0,6.0,0.00,55.0,10.00,7.0,NaN,NaN
57042,2018-01-01 04:51:00,NaN,CLR:00,8.0,-5.0,6.0,0.00,55.0,10.00,9.0,NaN,NaN


In [19]:
# clean H_Precipitation
clean_col = 'H_Precipitation'
weather_2018_df[clean_col].unique()
# weather_2018_df[weather_2018_df[clean_col].isnull() == True]
weather_2018_df[clean_col].fillna('0',inplace=True)
weather_2018_df[clean_col] = weather_2018_df[clean_col].apply(temp_func)
weather_2018_df[clean_col] = weather_2018_df[clean_col].astype('float64')
weather_2018_df[clean_col].unique()
for i in weather_2018_df[clean_col].unique():
    if check_str(i) != None:
        print(i)

In [20]:
weather_2018_df

,Date,H_PresentWeatherType,H_SkyConditions,H_DryBulbTemperature,H_DewPointTemperature,H_WetBulbTemperature,H_Precipitation,H_RelativeHumidity,H_Visibility,H_WindSpeed,Sunrise,Sunset
57038,2018-01-01 00:51:00,NaN,CLR:00,9.0,-4.0,7.0,0.0,55.0,10.00,9.0,NaN,NaN
57039,2018-01-01 01:51:00,NaN,CLR:00,9.0,-4.0,7.0,0.0,55.0,10.00,10.0,NaN,NaN
57040,2018-01-01 02:51:00,NaN,CLR:00,8.0,-4.0,6.0,0.0,57.0,10.00,10.0,NaN,NaN
57041,2018-01-01 03:51:00,NaN,CLR:00,8.0,-5.0,6.0,0.0,55.0,10.00,7.0,NaN,NaN
57042,2018-01-01 04:51:00,NaN,CLR:00,8.0,-5.0,6.0,0.0,55.0,10.00,9.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
62146,2018-05-31 21:51:00,BR:1 ||,OVC:08 3,65.0,64.0,64.0,0.0,97.0,3.00,0.0,NaN,NaN
62147,2018-05-31 22:51:00,BR:1 ||,OVC:08 3,66.0,64.0,65.0,0.0,93.0,3.00,0.0,NaN,NaN
62148,2018-05-31 23:51:00,BR:1 ||,OVC:08 2,65.0,64.0,64.0,0.0,97.0,3.00,0.0,NaN,NaN
62149,2018-05-31 23:59:00,NaN,NaN,65.0,64.0,64.0,0.0,NaN,NaN,NaN,427.0,1920.0


In [21]:
# clean H_Precipitation
clean_col = 'H_RelativeHumidity'
weather_2018_df[clean_col].fillna(method='ffill',inplace=True)
weather_2018_df[clean_col] = weather_2018_df[clean_col].apply(temp_func)
weather_2018_df[clean_col] = weather_2018_df[clean_col].astype('float64')
weather_2018_df[clean_col].unique()


array([ 55.,  57.,  60.,  53.,  48.,  44.,  40.,  37.,  35.,  34.,  39.,
        43.,  47.,  54.,  59.,  56.,  62.,  61.,  58.,  50.,  42.,  46.,
        52.,  51.,  49.,  45.,  41.,  28.,  31.,  32.,  75.,  81.,  84.,
        86.,  89.,  92.,  93.,  88.,  77.,  72.,  69.,  38.,  36.,  33.,
        66.,  70.,  67.,  68.,  73.,  76.,  79.,  71.,  74.,  83.,  90.,
        97.,  96.,  94., 100.,  85.,  78.,  63.,  30.,  29.,  82.,  87.,
        80.,  65.,  24.,  22.,  23.,  21.,  25.,  27.,  26.,  64.,  19.,
        16.,  15.,  20.,  18.,  17.,  14.,  13.,  12.,  91.])

In [22]:
def temp_func_V(x):
    try:
        float(x)
        return x
    except:
        if x[-1] == 'V':
            return x[:-1]

# clean H_Visibility
clean_col = 'H_Visibility'
weather_2018_df[clean_col].fillna(method='ffill',inplace=True)
weather_2018_df[clean_col] = weather_2018_df[clean_col].apply(temp_func_V)
weather_2018_df[clean_col] = weather_2018_df[clean_col].astype('float64')
weather_2018_df[clean_col].unique()

array([10.  ,  6.  ,  2.5 ,  1.75,  0.75,  0.5 ,  0.25,  0.  ,  5.  ,
        4.  ,  8.  ,  9.  ,  2.  ,  7.  ,  1.  ,  3.  ,  1.25,  1.5 ])

In [23]:
# clean H_WindSpeed
clean_col = 'H_WindSpeed'
# weather_2018_df[weather_2018_df[clean_col].isnull() == True]
weather_2018_df[clean_col].fillna('0',inplace=True)
weather_2018_df[clean_col] = weather_2018_df[clean_col].astype('float64')
weather_2018_df[clean_col].unique()


array([ 9., 10.,  7.,  6.,  8.,  3.,  5., 11.,  0., 15., 13., 14., 21.,
       17., 16., 18., 20.])

In [24]:
# clean 'Sunrise','Sunset'
clean_col = ['Sunrise','Sunset']
# weather_2018_df[weather_2018_df[clean_col].isnull() == True]
for c in clean_col:
    weather_2018_df[c].fillna(method='bfill',inplace=True)
    weather_2018_df[c].fillna(method='ffill',inplace=True)

## Cleaning Abbreviations

In [25]:
sky_types = {
    'CLR': 'clear sky',
    'FEW': 'few clouds',
    'SCT': 'scattered clouds',
    'BKN': 'broken clouds',
    'OVC': 'overcast'
}

def replace(x):
    try:
        for k,v in sky_types.items():
            if k in x:
                return v
    except:
        return x

# clean H_SkyConditions
clean_col = 'H_SkyConditions'
weather_2018_df[clean_col] = weather_2018_df[clean_col].apply(replace)
weather_2018_df[clean_col].fillna(method='ffill',inplace=True)
weather_2018_df[clean_col].unique()

array(['clear sky', 'broken clouds', 'overcast', 'scattered clouds',
       'few clouds'], dtype=object)

In [26]:
weather_types = {
    'DZ': 'Drizzle',
    'RA': 'Rain',
    'SN': 'Snow',
    'SG': 'Snow',
    'IC': 'Ice',
    'PL': 'Ice',
    'GR': 'Hail',
    'GS': 'Small Hail',
    'BR': 'Mist',
    'FG': 'Fog',
    'FU': 'Fog',
    'VA': 'Volcanic Ash',
    'DU': 'Dust',
    'SA': 'Sand',
    'HZ': 'Fog',
    'PY': 'Spray',
    'PO': 'Dust',
    'SQ': 'Squalls',
    'FC': 'Tornado',
    'SS': 'Sandstorm',
    'DS': 'Duststorm'
}

import re

def clean_weather_type(x):
    try:
        match = re.match('[-+]\w+', x)[0]
        if x[0] == '-':
            cond = 'light'
        if x[0] == '+':
            cond = 'heavy'
        for k,v in weather_types.items():
            if k in match:
                return v
    except:
        return 'clear'


# # clean H_PresentWeatherType
clean_col = 'H_PresentWeatherType'
weather_2018_df[clean_col] = weather_2018_df[clean_col].apply(clean_weather_type)
weather_2018_df[clean_col].fillna('clear',inplace=True)
weather_2018_df[clean_col].unique()

array(['clear', 'Snow', 'Rain'], dtype=object)

In [27]:
weather_2018_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5113 entries, 57038 to 62150
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Date                   5113 non-null   datetime64[ns]
 1   H_PresentWeatherType   5113 non-null   object        
 2   H_SkyConditions        5113 non-null   object        
 3   H_DryBulbTemperature   5113 non-null   float64       
 4   H_DewPointTemperature  5113 non-null   float64       
 5   H_WetBulbTemperature   5113 non-null   float64       
 6   H_Precipitation        5113 non-null   float64       
 7   H_RelativeHumidity     5113 non-null   float64       
 8   H_Visibility           5113 non-null   float64       
 9   H_WindSpeed            5113 non-null   float64       
 10  Sunrise                5113 non-null   float64       
 11  Sunset                 5113 non-null   float64       
dtypes: datetime64[ns](1), float64(9), object(2)
memory usage:

In [31]:
weather_2018_df.to_csv('../Datasets/NYC_Weather_Clean.csv',index=False)